In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import cm
from matplotlib.colors import LogNorm
import seaborn as sns
import statsmodels.api as sm
import re
import os

In [ ]:
def initializeDataFrame(dirName='.',
                        snapshotDir='movies/snapshots',
                        params=['pf','sp','lp'],
                        analyses=['global_order']):
    """Generates dataframe of all simcore analysis files, assuming
    file naming convention of containing the substrings listed in the
    params list, followed by the parameter quantity and an underscore.
    This function tabulates these quantities using the file names of any
    bitmaps (final state snapshots) found in snapshotDir. It then looks
    for the corresponding analyses files whose extension is given by the
    substrings found in the analyses list.
    """
    df=pd.DataFrame(columns=params+['snapshots']+analyses)
    df['snapshots'] = createFileList('bmp',snapshotDir)
    for i in analyses:
        df[i]=createFileList(i,dirName)
    for i in df.index:
        for j in params:
            fname = df.iloc[i][analyses[0]]
            fname = fname[-fname[::-1].find('/'):]
            try:
                start = fname.find(j)+len(j)
                end = fname[start:].find('_')
            except:
                print(fname)
            df.iloc[i][j] = float(fname[start:start+end])
    return df


def createFileList(extension,dirName='.'):
    """Returns a pandas series consisting of all files with the provided
    extension in directory dirName, defaulting to the current working
    directory.
    """
    assert isinstance(dirName, str), (
            "The 'dirName' argument must be a string!")
    assert isinstance(extension, str), (
            "The 'extension' argument must be a string!")
    file_names = [dirName+'/'+i for i in os.listdir(dirName) 
                  if i[-len(extension):] == extension]
    if len(file_names)==0:
        print("No '" + extension + "' files found in directory '"
              + dirName + "'.")
        return
    file_names.sort()
    return pd.Series(file_names)

In [ ]:
def GetAnalysisGridPlotHandles(df,analyze):
    """Provides the figure and axis handles for a plot grid that are
    determined by the number of unique persistence lengths (lps) and
    U_max's (sps). The dataframe provided must only have one packing
    fraction (pf).
    
    Returns fig, ax, len(sps), len(lps), sps, lps, where the subplot
    dimensions are len(lps) rows and len(sps) columns sps and lps are
    unique, sorted ascending sp and lp entries in dataframe df. 
    """
    assert isinstance(df,pd.DataFrame), (
            "Parameter 'df' must be a pandas dataframe!")
    assert isinstance(analyze,str), (
            "Parameter 'analyze' must be a string!")
    if ('pf' not in df.columns 
        or 'sp' not in df.columns
        or 'lp' not in df.columns):
        print("Dataframe df is missing parameter columns.")
        return None
    if analyze not in df.columns:
        print("Dataframe df is missing "+analyze+" column.")
        return None
    if (len(df['pf'].unique()) != 1):
        print("Dataframe df must only have one unique packing fraction.")
        return None
    is_local = (analyze[:5] == 'local')
    sps = np.sort(df.sp.unique())
    lps = np.sort(df.lp.unique())
    fig_x = len(sps)
    fig_y = len(lps)
    if (fig_x == 0 or fig_y == 0):
        print("Dataframe df is missing entries in sp and/or lp columns.")
        return None
    figscale = (8,6)
    is_not_snapshots=True
    if (analyze == 'snapshots'):
        figscale = (8,8)
        is_not_snapshots=False
    fig,ax = plt.subplots(fig_y, fig_x, 
                          figsize=(figscale[0]*fig_x,figscale[1]*fig_y),
                          sharex=is_local, sharey=is_not_snapshots)
    return fig, ax, fig_x, fig_y, sps, lps

In [ ]:
def createSnapshotPlots(df,dirName='.'):
    """Generates a grid of plots displaying snapshot images given by the
    files in the 'snapshots' column of the dataframe, with the images
    arranged to display the effect due to increasing U_max (x-axis) and 
    increasing persistence length/length ratio (y-axis).
    """
    assert isinstance(dirName,str), "'dirName' must be a string!"
    if dirName[-1]=='/':
        dirName=dirName[:-1]
    fig,ax,fig_x,fig_y,sps,lps=GetAnalysisGridPlotHandles(df,'snapshots')
    pf = df.iloc[0].pf
    for i in df.index:
        icol=np.where(sps==df.iloc[i].sp)[0][0]
        irow=fig_y-np.where(lps==df.iloc[i].lp)[0][0]-1
        a = ax[irow][icol]
        a.imshow(mpimg.imread(df.iloc[i].snapshots))
        a.set_yticks([])
        a.set_xticks([])
        a.locator_params(nbins=1)
    for irow,lp in zip(range(fig_y),lps[::-1]):
        ax[irow][0].set_yticks([400])
        ax[irow][0].set_yticklabels([int(lp)],fontsize=50)
        ax[irow][0].tick_params(length=10,width=5)
    for icol,sp in zip(range(fig_x),sps):
        ax[fig_y-1][icol].set_xticks([400])
        ax[fig_y-1][icol].set_xticklabels([int(sp)],fontsize=50)
        ax[fig_y-1][icol].tick_params(length=10,width=5)
    fig.tight_layout(pad=15,h_pad=0,w_pad=-5)
    fig.text(0.5, 0.025, r'$U_{max}/k_B T$', ha='center',fontsize=70)
    fig.text(0.025, 0.5, r'$L_p/L$', va='center',
             rotation='vertical',fontsize=70)
    fig.suptitle("Final state snapshot, "+str(100*pf)
                 +"% pf",fontsize=70,y=0.95)

In [ ]:
def createGlobalOrderPlots(df,saveDirName="."):
    """Generates two grids of plots displaying time series of the global
    order parameters, including the global polar/nematic order on one
    figure and global spiral number/spiral handedness on a second figure.
    Grid of plots are arranged to display the effect due to increasing
    U_max (x-axis) and increasing persistence length/length ratio (y-axis).
    """
    assert isinstance(saveDirName,str), "'dirName' must be a string!"
    analyze = 'global_order'
    if saveDirName[-1]=='/': 
        saveDirName=saveDirName[:-1]   

    # First, we're going to plot nematic and polar global order params
    fig,ax,fig_x,fig_y,sps,lps = GetAnalysisGridPlotHandles(df,analyze)
    pf = df.iloc[0].pf
    for i in df.index:
        icol=np.where(sps==df.iloc[i].sp)[0][0]
        irow=fig_y-np.where(lps==df.iloc[i].lp)[0][0]-1
        a = ax[irow][icol]
        fname=df.iloc[i][analyze]
        goDF=GetGlobalOrderDF(fname)
        goDF.plot(y="nematic_order_mag",color='blue',linewidth=3,ax=a)
        goDF.plot(y="polar_order_mag",color='red',linewidth=3,ax=a)
        a.set_xlabel('')
        a.legend(['Nematic order','Polar order'],fontsize=20)
        a.tick_params(length=5,width=2.5,labelsize=20)
    for irow in range(fig_y): 
        ax[irow][0].set_ylabel('Order parameter',fontsize=30)
    for icol in range(fig_x): 
        ax[fig_y-1][icol].set_xlabel('Time',fontsize=30)
    fig.tight_layout(pad=15,h_pad=0,w_pad=0)
    fig.text(0.5, 0.05,
             r'$\Longrightarrow U_{max}/k_B T \Longrightarrow$',
             ha='center',fontsize=70)
    fig.text(0.01, 0.5,
             r'$\Longrightarrow L_p/L \Longrightarrow$',
             va='center', rotation='vertical',fontsize=70)
    fig.suptitle("Global Polar/Nematic Order, "+str(100*pf)+"% pf",
                 fontsize=70,y=0.95)
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_global_order_plots.png",dpi=300)
    
    # Now do the same for spiral order parameters
    fig,ax,fig_x,fig_y,sps,lps = GetAnalysisGridPlotHandles(df,analyze)
    for i in df.index:
        icol=np.where(sps==df.iloc[i].sp)[0][0]
        irow=fig_y-np.where(lps==df.iloc[i].lp)[0][0]-1
        a = ax[irow][icol]
        fname=df.iloc[i][analyze]
        goDF=GetGlobalOrderDF(fname)
        goDF.plot(y="spiral_order",color='blue',linewidth=3,ax=a)
        goDF.plot(y="signed_spiral_order",color='red',linewidth=3,ax=a)
        a.set_xlabel('')
        a.legend(['Spiral order','Spiral handedness'],fontsize=20)
        a.tick_params(length=5,width=2.5,labelsize=20)
    for irow in range(fig_y): 
        ax[irow][0].set_ylabel('Order parameter',fontsize=30)
    for icol in range(fig_x): 
        ax[fig_y-1][icol].set_xlabel('Time',fontsize=30)
    fig.tight_layout(pad=15,h_pad=0,w_pad=0)
    fig.text(0.5, 0.05,
             r'$\Longrightarrow U_{max}/k_B T \Longrightarrow$',
             ha='center',fontsize=70)
    fig.text(0.01, 0.5,
             r'$\Longrightarrow L_p/L \Longrightarrow$',
             va='center', rotation='vertical',fontsize=70)
    fig.suptitle("Spiral Order, "+str(100*pf)+"% pf",fontsize=70,y=0.95)
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_spiral_order_plots.png",dpi=300)


def CalculateGlobalOrderMagnitudes(df):
    """Given a global order dataframe, calculates magnitude of polar
    order vector and maximum eigenvalue of nematic order tensor.
    """
    df['polar_order_mag'] = np.sqrt(df.polar_order_x**2 
                                    + df.polar_order_y**2 
                                    + df.polar_order_z**2)
    df['nematic_order_mag'] = df.apply(lambda x: maxEig(
        x['nematic_order_xx'],
        x['nematic_order_xy'],
        x['nematic_order_yx'],
        x['nematic_order_yy']), axis=1)

    
def maxEig(xx,xy,yx,yy):
    """Returns the max eigenvalue of 2D matrix with elements xx, xy,
    yx, yy.
    """
    return max(np.linalg.eig(np.array([[xx,xy],[yx,yy]]))[0])

    
def GetGlobalOrderDF(fname):
    """Calculates time series of global orders parameters (polar order
    vector magnitude and maximum eigenvalues of nematic order tensor Q)
    from .global_order file with name 'fname' and returns global order
    dataframe.
    """
    assert isinstance(fname,str), "'fname' must be a string!"
    df = pd.read_csv(fname,delim_whitespace=True,skiprows=1,
                     index_col='time')
    CalculateGlobalOrderMagnitudes(df)
    return df


In [ ]:
def createLocalOrderPlots(df, saveDirName=".", lo_width=20,
                          colorMap=cm.viridis, vlims=(0,3)):
    """Runs createLocalPDFPlot (with the provided vlims),
    createLocalNematicPlot, and createLocalPolarPlot (with default
    vlims between zero and one) in secession, with local order width
    given by lo_width. Each of these function generates a grid of
    plots displaying histograms of the pair distribution function,
    nematic orientation correlation function, and polar order
    correlation functions, respectively. The plots are arranged to
    display the effect due to increasing U_max (x-axis) and increasing
    persistence length/length ratio (y-axis).
    """
    createLocalPDFPlot(df,saveDirName,lo_width,colorMap,vlims)
    createLocalNematicPlot(df,saveDirName,lo_width,colorMap,(0,1))
    createLocalPolarPlot(df,saveDirName,lo_width,colorMap,(0,1))

    
def createLocalPDFPlot(df, saveDirName=".", lo_width=20,
                       colorMap=cm.viridis, vlims=(0,1)):
    """Generates a grid of plots displaying histograms of the pair 
    distribution functions, with the plots arranged to display the
    effect due to increasing U_max (x-axis) and increasing persistence 
    length/length ratio (y-axis).
    """
    assert isinstance(saveDirName,str), "'dirName' must be a string!"
    analyze = 'local_pdf'
    if saveDirName[-1]=='/': 
        saveDirName=saveDirName[:-1]
    fig,ax,fig_x,fig_y,sps,lps = GetAnalysisGridPlotHandles(df,analyze)
    fig.tight_layout(pad=15,h_pad=2,w_pad=2)
    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    pf = df.iloc[0].pf
    for i in df.index:
        icol=np.where(sps==df.iloc[i].sp)[0][0]
        irow=fig_y-np.where(lps==df.iloc[i].lp)[0][0]-1
        a = ax[irow][icol]
        fname=df.iloc[i][analyze]
        poDF=pd.read_csv(fname,sep=" ",header=None)
        poDF=poDF.dropna(axis=1)
        sns.heatmap(poDF,vmin=vlims[0], vmax=vlims[1], cmap=colorMap,
                    ax=a, cbar_ax=cbar_ax)
        a.set_xlabel('')
        a.tick_params(length=0, width=0, labelsize=20)
        a.locator_params(nbins=0)
    for irow in range(fig_y): 
        a=ax[irow][0]
        a.set_ylabel('y',fontsize=30)
        a.locator_params(axis='y',nbins=11)
        a.set_yticklabels(-np.round(np.arange(
            -0.5*lo_width,0.5*lo_width+lo_width/10,lo_width/10),1))
        a.tick_params(axis='y',length=5,width=2.5,labelsize=20)
    for icol in range(fig_x): 
        a=ax[fig_y-1][icol]
        a.set_xlabel('x',fontsize=30)
        a.locator_params(axis='x',nbins=11)
        ticks=a.get_xticks()
        a.set_xticks(ticks+min(abs(ticks-100)))
        a.set_xticklabels(map(round,list((ticks-100)*20/200)))
        a.tick_params(axis='x',length=5,width=2.5,labelsize=20)
    cbar_ax.tick_params(length=10,width=5,labelsize=30)
    fig.text(0.45, 0.01,
             r'$\Longrightarrow U_{max}/k_B T \Longrightarrow$',
             ha='center',fontsize=70)
    fig.text(0.0, 0.5, r'$\Longrightarrow L_p/L \Longrightarrow$',
             va='center', rotation='vertical',fontsize=70)
    fig.suptitle("Pair Distribution Function, "+str(100*pf)+"% pf",
                 fontsize=70,y=0.95)
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_local_pdf_plots.png",dpi=300)
    
    
def createLocalNematicPlot(df, saveDirName=".", lo_width=20,
                           colorMap=cm.viridis, vlims=(0,1)):
    """Generates a grid of plots displaying histograms of the nematic 
    orientation correlation functions, with the plots arranged to display
    the effect due to increasing U_max (x-axis) and increasing persistence 
    length/length ratio (y-axis).
    """
    assert isinstance(saveDirName,str), "'dirName' must be a string!"
    analyze = 'local_nematic'
    if saveDirName[-1]=='/': 
        saveDirName=saveDirName[:-1]
    fig,ax,fig_x,fig_y,sps,lps = GetAnalysisGridPlotHandles(df,analyze)
    fig.tight_layout(pad=15,h_pad=2,w_pad=2)
    fig.subplots_adjust(right=0.8)
    pf = df.iloc[0].pf
    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    for i in df.index:
        icol=np.where(sps==df.iloc[i].sp)[0][0]
        irow=fig_y-np.where(lps==df.iloc[i].lp)[0][0]-1
        a = ax[irow][icol]
        fname=df.iloc[i][analyze]
        poDF=pd.read_csv(fname,sep=" ",header=None)
        poDF=poDF.dropna(axis=1)
        sns.heatmap(poDF,vmin=vlims[0],vmax=vlims[1],
                    cmap=colorMap,ax=a,cbar_ax=cbar_ax)
        a.set_xlabel('')
        a.tick_params(length=0,width=0,labelsize=20)
        a.locator_params(nbins=0)
    for irow in range(fig_y): 
        a=ax[irow][0]
        a.set_ylabel('y',fontsize=30)
        a.locator_params(axis='y',nbins=11)
        a.set_yticklabels(-np.round(np.arange(
            -0.5*lo_width,0.5*lo_width+lo_width/10,lo_width/10),1))
        a.tick_params(axis='y',length=5,width=2.5,labelsize=20)
    for icol in range(fig_x): 
        a=ax[fig_y-1][icol]
        a.set_xlabel('x',fontsize=30)
        a.locator_params(axis='x',nbins=11)
        ticks=a.get_xticks()
        a.set_xticks(ticks+min(abs(ticks-100)))
        a.set_xticklabels(map(round,list((ticks-100)*20/200)))
        a.tick_params(axis='x',length=5,width=2.5,labelsize=20)
    cbar_ax.tick_params(length=10,width=5,labelsize=30)
    fig.text(0.45, 0.01,
             r'$\Longrightarrow U_{max}/k_B T \Longrightarrow$',
             ha='center',fontsize=70)
    fig.text(0.0, 0.5, r'$\Longrightarrow L_p/L \Longrightarrow$',
             va='center', rotation='vertical',fontsize=70)
    fig.suptitle("Nematic Orientational Correlation, "+str(100*pf)+"% pf",
                 fontsize=70,y=0.95)
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_local_nematic_plots.png",dpi=300)
    
    
def createLocalPolarPlot(df, saveDirName=".", lo_width=20,
                         colorMap=cm.viridis, vlims=(0,1)):
    """Generates a grid of plots displaying histograms of the polar 
    orientation correlation functions, with the plots arranged to display
    the effect due to increasing U_max (x-axis) and increasing persistence 
    length/length ratio (y-axis).
    """
    assert isinstance(saveDirName,str), "'dirName' must be a string!"
    analyze = 'local_polar'
    if saveDirName[-1]=='/': 
        saveDirName=saveDirName[:-1]
    fig,ax,fig_x,fig_y,sps,lps = GetAnalysisGridPlotHandles(df,analyze)
    fig.tight_layout(pad=15,h_pad=2,w_pad=2)
    fig.subplots_adjust(right=0.8)
    pf = df.iloc[0].pf
    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    for i in df.index:
        icol=np.where(sps==df.iloc[i].sp)[0][0]
        irow=fig_y-np.where(lps==df.iloc[i].lp)[0][0]-1
        a = ax[irow][icol]
        fname=df.iloc[i][analyze]
        poDF=pd.read_csv(fname,sep=" ",header=None)
        poDF=poDF.dropna(axis=1)
        sns.heatmap(poDF,vmin=vlims[0],vmax=vlims[1],
                    cmap=colorMap,ax=a,cbar_ax=cbar_ax)
        a.set_xlabel('')
        a.tick_params(length=0,width=0,labelsize=20)
        a.locator_params(nbins=0)
    for irow in range(fig_y): 
        a=ax[irow][0]
        a.set_ylabel('y',fontsize=30)
        a.locator_params(axis='y',nbins=11)
        a.set_yticklabels(-np.round(np.arange(
            -0.5*lo_width,0.5*lo_width+lo_width/10,lo_width/10),1))
        a.tick_params(axis='y',length=5,width=2.5,labelsize=20)
    for icol in range(fig_x): 
        a=ax[fig_y-1][icol]
        a.set_xlabel('x',fontsize=30)
        a.locator_params(axis='x',nbins=11)
        ticks=a.get_xticks()
        a.set_xticks(ticks+min(abs(ticks-100)))
        a.set_xticklabels(map(round,list((ticks-100)*20/200)))
        a.tick_params(axis='x',length=5,width=2.5,labelsize=20)
    cbar_ax.tick_params(length=10,width=5,labelsize=30)
    fig.text(0.45, 0.01,
             r'$\Longrightarrow U_{max}/k_B T \Longrightarrow$',
             ha='center',fontsize=70)
    fig.text(0.0, 0.5, r'$\Longrightarrow L_p/L \Longrightarrow$',
             va='center', rotation='vertical',fontsize=70)
    fig.suptitle("Polar Orientational Correlation, "+str(100*pf)+"% pf",
                 fontsize=70,y=0.95)
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_local_polar_plots.png",dpi=300)
    

In [ ]:
def createOrientationCorrPlots(df,saveDirName="."):
    """Generates a grid of plots displaying histograms of the instant
    bond overlap and filament overlap counts as a time series, with the
    plots arranged to display the effect due to increasing U_max (x-axis)
    and increasing persistence length/length ratio (y-axis).
    
    Also generates a second figure of two plots that quantify the
    characteristic orientation decorrelation time as functions of both
    U_max and Lp/L.
    """
    assert isinstance(saveDirName,str), "'dirName' must be a string!"
    analyze = 'orientation_corr'
    if saveDirName[-1]=='/': 
        saveDirName=saveDirName[:-1]
    fig,ax,fig_x,fig_y,sps,lps = GetAnalysisGridPlotHandles(df,analyze)
    pf = df.iloc[0].pf
    taus = np.zeros((fig_y,fig_x))
    tau_errors = np.zeros((fig_y,fig_x))
    for i in df.index:
        icol=np.where(sps==df.iloc[i].sp)[0][0]
        irow=fig_y-np.where(lps==df.iloc[i].lp)[0][0]-1
        a = ax[irow][icol]
        fname=df.iloc[i][analyze]
        ocDF = pd.read_csv(fname,skiprows=1,delim_whitespace=True)
        ocDF = ocDF[:ocDF.time.size-ocDF.time.iloc[-1]-1]
        gb=ocDF.groupby('time')
        x=ocDF.time.unique()
        y=gb.orientation_corr_avg.mean()
        ysem=gb.orientation_corr_sem.mean()
        model=sm.GLM(y, x,
                     family=sm.families.Gaussian(sm.families.links.log))
        fit=model.fit()
        tau=-1.0/fit.params.x1
        taus[irow][icol] = tau
        tau_errors[irow][icol] = tau**2 * fit.bse.x1
        a.errorbar(x,y,yerr=ysem)
        theory = lambda t: np.exp(-t/tau)
        a.plot(x,theory(x),'r')
        a.set_ylabel(r"$\langle u(0)\cdot u(t)\rangle$",fontsize=18)
        a.set_xlabel("Time (t)",fontsize=18)
        a.legend([r"Fit, $\tau=%2.2f$" % tau,'Simulation'],fontsize=16)
        a.grid(True,linestyle='--')
    for irow in range(fig_y): 
        ax[irow][0].set_ylabel(
            r'$\langle u(t)\cdot u(t+\tau) \rangle$',fontsize=30)
    for icol in range(fig_x): 
        ax[fig_y-1][icol].set_xlabel('Time',fontsize=30)
    fig.tight_layout(pad=15,h_pad=0,w_pad=0)
    fig.text(0.5, 0.05,
             r'$\Longrightarrow U_{max}/k_B T \Longrightarrow$',
             ha='center',fontsize=70)
    fig.text(0.01, 0.5,
             r'$\Longrightarrow L_p/L \Longrightarrow$',
             va='center', rotation='vertical',fontsize=70)
    fig.suptitle("Orientation autocorrelation, "+str(100*pf)+"% pf",fontsize=70,y=0.95)
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_orientation_corr_plots.png",dpi=300)
    fig,ax=plt.subplots(1,2,figsize=(16,6))
    for i in range(taus.shape[1]):
        ax[0].errorbar(lps,taus[:,i],yerr=tau_errors[:,i],
                       capsize=2,capthick=2)
    for i in range(taus.shape[0]):
        ax[1].errorbar(sps,taus[i,:],yerr=tau_errors[i,:],
                       capsize=2,capthick=2)
    for a in ax:
        a.grid(True,linestyle='--')
        a.set_ylabel(r'$\tau$',fontsize=30)
        a.tick_params(labelsize=16)
    ax[0].legend([r'$U_{max}/k_B T=$'+str(i) for i in sps],fontsize=20)
    ax[0].set_xlabel(r'$L_p/L$',fontsize=30)
    ax[1].set_xlabel(r'$U_{max}/k_B T$',fontsize=30)
    ax[1].legend([r'$L_p/L=$'+str(i) for i in lps],fontsize=20)
    fig.suptitle(r'Orientation decorrelation time $\tau$, '+str(100*pf)+'% pf',
                 fontsize=30)
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_orientation_corr_graph_plots.png",dpi=300)


In [ ]:
def createPolarOrderPlots(df,saveDirName=".",contact_cut=10,
                          colorMap=cm.viridis,vlims=(1e-7,1e-1)):
    """Generates a grid of plots displaying histograms of the local
    polar order as a function of contact number, with the plots arranged
    to display the effect due to increasing U_max (x-axis) and increasing
    persistence length/length ratio (y-axis). The parameter contact_cut
    is used to renormalize the x-axis ticklabels to be in the range
    (0,contact_cut).
    """
    assert isinstance(saveDirName,str), "'dirName' must be a string!"
    analyze = 'polar_order'
    if saveDirName[-1]=='/': 
        saveDirName=saveDirName[:-1]
    fig,ax,fig_x,fig_y,sps,lps = GetAnalysisGridPlotHandles(df,analyze)
    fig.tight_layout(pad=15,h_pad=0,w_pad=0)
    fig.subplots_adjust(right=0.8)
    pf = df.iloc[0].pf
    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    for i in df.index:
        icol=np.where(sps==df.iloc[i].sp)[0][0]
        irow=fig_y-np.where(lps==df.iloc[i].lp)[0][0]-1
        a = ax[irow][icol]
        fname=df.iloc[i][analyze]
        poDF=pd.read_csv(fname,skiprows=2,delim_whitespace=True,
                         header=None)
        poDF=poDF.dropna(axis=1)
        data = poDF.replace(0,1)
        data=data/data.sum().sum()
        min_data = data.min().min()
        if (min_data == 0):
            min_data = 1
        max_data = data.max().max()
        log_norm = LogNorm(vmin=min_data, vmax=max_data)
        cbar_ticks = [10**i for i in 
                      range(int(np.floor(np.log10(min_data))),
                            1 + int(np.ceil(np.log10(max_data))))]
        sns.heatmap(data, vmin=vlims[0],vmax=vlims[1],norm=log_norm,
                    cmap=cm.viridis, ax=a,
                    cbar_kws={"ticks": cbar_ticks}, cbar_ax=cbar_ax)
        a.set_xlabel('')
        a.tick_params(length=0,width=0,labelsize=20)
        a.set_xticklabels([])
        a.set_yticklabels([])
    for irow in range(fig_y): 
        a=ax[irow][0]
        a.set_ylabel(r'Local Polar Order, $p_i$',fontsize=30)
        a.locator_params(axis='y',nbins=21)
        a.tick_params(axis='y',length=5,width=2.5,labelsize=20)
        ticks=a.get_yticks()
        a.set_yticks(ticks+min(abs(ticks-25)))
        k=list(-(ticks/50-1))
        a.set_yticklabels(np.round(k-min([abs(i) for i in k]),2))
    for icol in range(fig_x): 
        a=ax[fig_y-1][icol]
        a.set_xlabel(r'Contact Number, $c_i$',fontsize=30)
        a.locator_params(axis='x',nbins=21)
        a.tick_params(axis='x',length=5,width=2.5,labelsize=20)
        ticks=a.get_xticks()
        a.set_xticklabels(
            np.round(list((ticks-min(ticks))*contact_cut/100),1))
    cbar_ax.tick_params(length=10,width=5,labelsize=30)
    fig.text(0.45, 0.01,
             r'$\Longrightarrow U_{max}/k_B T \Longrightarrow$',
             ha='center',fontsize=70)
    fig.text(0.0, 0.5, r'$\Longrightarrow L_p/L \Longrightarrow$',
             va='center', rotation='vertical',fontsize=70)
    fig.suptitle("Local Polar Order, "+str(100*pf)+"% pf",fontsize=70,y=0.95)
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_polar_order_plots.png",dpi=300)


In [ ]:
def createOverlapPlots(df,saveDirName="."):
    """Generates a grid of plots displaying histograms of the instant
    bond overlap and filament overlap counts as a time series, with the
    plots arranged to display the effect due to increasing U_max (x-axis)
    and increasing persistence length/length ratio (y-axis).
    
    Also generates a second plot that quantifies the overlap initiation
    rate as a function of U_max and with Lp/L as multiple plot lines.
    """
    assert isinstance(saveDirName,str), "'dirName' must be a string!"
    analyze = 'overlaps'
    if saveDirName[-1]=='/':
        saveDirName=saveDirName[:-1]
    
    # Generate time series of instantaneous bond and filament overlaps.
    # Also, use the time series data of filament overlap initiations
    # (which is very linear) to determine their rate.
    fig,ax,fig_x,fig_y,sps,lps = GetAnalysisGridPlotHandles(df,analyze)
    pf = df.iloc[0].pf
    init_rates=np.empty((fig_y,fig_x),dtype=object)
    rate_errors=np.empty((fig_y,fig_x),dtype=object)
    for i in df.index:
        icol=np.where(sps==df.iloc[i].sp)[0][0]
        irow=fig_y-np.where(lps==df.iloc[i].lp)[0][0]-1
        a = ax[irow][icol]
        fname=df.iloc[i][analyze]
        goDF=pd.read_csv(fname,delim_whitespace=True)
        goDF['n_tangled'] = (goDF['n_total_crossings_init'] 
                             - goDF['n_total_crossings_complete'])
        goDF.plot(x='time',y="n_instant_bond_overlaps",
                  color='blue',linewidth=3,ax=a)
        goDF.plot(x='time',y="n_tangled",color='red',linewidth=3,ax=a)
        begin=int(goDF['time'].size/10)
        model=sm.OLS(goDF['n_total_crossings_init'].iloc[begin:],
                     sm.add_constant(goDF['time'].iloc[begin:]))
        model=model.fit()
        init_rates[irow][icol]=model.params[1]
        rate_errors[irow][icol]=model.bse[1]
        a.set_xlabel('')
        a.legend(['Bond overlaps','Filament overlaps'],fontsize=20)
        a.tick_params(length=5,width=2.5,labelsize=20)
        a.ticklabel_format(style='sci', axis='y', scilimits=(0,0),
                           useMathText=True)
        a.yaxis.get_offset_text().set_fontsize(20)
    for irow in range(fig_y): 
        ax[irow][0].set_ylabel('Overlap number',fontsize=30)
    for icol in range(fig_x): 
        ax[fig_y-1][icol].set_xlabel('Time',fontsize=30)
    fig.tight_layout(pad=15,h_pad=0,w_pad=0)
    fig.text(0.5, 0.05,
             r'$\Longrightarrow U_{max}/k_B T \Longrightarrow$',
             ha='center',fontsize=70)
    fig.text(0.01, 0.5, r'$\Longrightarrow L_p/L \Longrightarrow$',
             va='center', rotation='vertical',fontsize=70)
    fig.suptitle("Filament overlap events, "+str(100*pf)+"% pf",fontsize=70,
                 y=0.95)
    fig.savefig(saveDirName+"/pf"+str(pf)+"_overlap_plots.png",dpi=300)
    
    # Now plot the overlap initiation rates as a funciton of U_max,
    # with each Lp/L on its own line.
    fig,ax=plt.subplots(1,1,figsize=(8,6))
    for i in range(init_rates.shape[0]):
        ax.errorbar(sps,init_rates[i,:],yerr=rate_errors[i,:],capsize=2,
                    capthick=2)
    ax.grid(True,linestyle='--')
    ax.set_ylabel(r'$f_o$',fontsize=30)
    ax.tick_params(labelsize=16)
    ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0),
                        useMathText=True)
    ax.yaxis.get_offset_text().set_fontsize(16)
    ax.set_xlabel(r'$U_{max}/k_B T$',fontsize=30)
    ax.legend([r'$L_p/L=$'+str(i) for i in lps],fontsize=20)
    fig.suptitle(r'Overlap initation rate $f_o$, '+str(100*pf)
                 +'% pf',fontsize=25)
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_overlap_rate_plot.png",dpi=300)


In [ ]:
def createAnalysisPlots(df,saveDirName='.'):
    """This function runs the following functions in secession:
    createSnapshotPlots, createGlobalOrderPlots, createLocalOrderPLots,
    createPolarOrderPlots, createOrientationCorrPlots, createOverlapPlots.
    In addition, only function defaults are called, except for the passed
    dataframe and the directory where all the plots are saved.
    """
    createSnapshotPlots(df,saveDirName)
    createGlobalOrderPlots(df,saveDirName)
    createLocalOrderPlots(df,saveDirName)
    createPolarOrderPlots(df,saveDirName)
    createOrientationCorrPlots(df,saveDirName)
    createOverlapPlots(df,saveDirName)

In [ ]:
df=initializeDataFrame(dirName="./pf0.2/condensed_results",
                           snapshotDir="./pf0.2/movies/snapshots/",
                           params=['pf', 'sp', 'lp'],
                           analyses=[
                               'global_order',
                               'overlaps',
                               'orientation_corr',
                               'local_pdf',
                               'local_nematic',
                               'local_polar',
                               'polar_order'
                           ])

df.iloc[7].sp = 25
df.iloc[11].sp = 50
df = df.drop([0,6,10])
df = df.reset_index(drop=True)

In [ ]:
df=df[
    (df['lp']!=20) 
    & (df['sp']!=25) 
    & (df['sp']!=15)
    ].reset_index(drop=True)

In [ ]:
createAnalysisPlots(df)

In [ ]:
analyze = 'local_pdf'
# if saveDirName[-1]=='/': 
#     saveDirName=saveDirName[:-1]
# fig,ax,fig_x,fig_y,sps,lps = GetAnalysisGridPlotHandles(df,analyze)
# fig.tight_layout(pad=15,h_pad=2,w_pad=2)
# fig.subplots_adjust(right=0.8)
# cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
pf = df.iloc[0].pf
# for i in df.index:
#     icol=np.where(sps==df.iloc[i].sp)[0][0]
#     irow=fig_y-np.where(lps==df.iloc[i].lp)[0][0]-1
#     a = ax[irow][icol]
#     fname=df.iloc[i][analyze]
#     poDF=pd.read_csv(fname,sep=" ",header=None)
#     poDF=poDF.dropna(axis=1)

In [ ]:
analyze = 'local_pdf'
fname=df.iloc[11][analyze]
poDF=pd.read_csv(fname,sep=" ",header=None)
poDF=poDF.dropna(axis=1)
def foldTimeSeries(ts):
    assert isinstance(ts,(np.ndarray,pd.Series)), (
        "ts must be of pandas Series or numpy ndarray type")
    if isinstance(ts,pd.Series):
        return 0.5 * (ts[:len(ts)//2][::-1].reset_index(drop=True)
                      + ts[len(ts)//2:].reset_index(drop=True))
    elif isinstance(ts, np.ndarray):
        return 0.5 * (ts[:len(ts)//2][::-1] + ts[len(ts)//2:])
    else:
        print("Time series datatype unrecognized in foldTimeSeries!")

In [ ]:
plt.figure()
x = np.linspace(-10,10, len(y_avg))
y_avg = poDF.mean(axis=0)
midpoint = poDF.shape[0]//2
y_mid = poDF.iloc[midpoint]
#plt.plot(x, y_avg)
plt.plot(x, y_mid)

y_mid_fold = foldTimeSeries(y_mid)
y_avg_fold = foldTimeSeries(y_avg)
x_fold = np.linspace(0, 10, len(y_mid_fold))
plt.plot(x_fold, y_mid_fold)
#plt.plot(x_fold, y_avg_fold)
plt.xlim(0,10)
#plt.ylim(0,1)
plt.grid(True, linestyle='dashed')
plt.xlabel('r')
plt.ylabel('g(r)')
plt.show()

In [ ]:
y = y_mid_fold-1
# Number of sample points
N = len(y)
# sample spacing
T = 1/10
yf = np.fft.fft(y)
xf = np.linspace(0.0, 1.0/(2.0*T), N//2)
yplot = 2.0/N * np.abs(yf[0:N//2])
dyplot = np.gradient(yplot)
for i,j in enumerate(dyplot):
    if j < 0 and dyplot[i+1] > 0:
        break
k = yplot[i:].argmax()
klim = yplot[i:][k]
xmax = recenteringFunction(yplot[i+k], yplot[i+k-1], yplot[i+k+1],
                        xf[i+k], xf[i+k-1], xf[i+k+1]) 
plt.vlines(xmax, ymin=0, ymax=1.1*klim,
           color='red', linestyles='dashed')
print(xmax)
#2.0/N * np.abs(yf[1:N//2])
#yplot = 1 + 1/250 * yplot
plt.plot(xf[1:], yplot[1:])
plt.grid(True, linestyle='dashed')

In [ ]:
def recenteringFunction(y_mx, y_2nd, y_3rd, x_mx, x_2nd, x_3rd):
    assert (y_mx >= y_2nd and y_mx >= y_3rd), (
        "y_mx must be greater than or equal to y_2nd and y_3rd")
    # Ensure that y_2nd >= y_3rd
    if (y_3rd > y_2nd):
        temp = y_2nd
        y_2nd = y_3rd
        y_3rd = temp
        temp = x_2nd
        x_2nd = x_3rd
        x_3rd = temp
    # If it is the case that y_mx - y_3rd is zero, then we must
    # also have the case that y_mx == y_2nd == y_3rd
    if (y_mx - y_3rd == 0):
        return x_mx
    weight = (y_mx-y_2nd)**2 / (y_mx-y_3rd)**2
    return (1-weight)*(x_mx+x_2nd)/2 + weight*x_mx

In [ ]:
fig,ax,fig_x,fig_y,sps,lps = GetAnalysisGridPlotHandles(df,analyze)
pf = df.iloc[0].pf
#init_rates=np.empty((fig_y,fig_x),dtype=object)
#rate_errors=np.empty((fig_y,fig_x),dtype=object)
for i in df.index:
    icol=np.where(sps==df.iloc[i].sp)[0][0]
    irow=fig_y-np.where(lps==df.iloc[i].lp)[0][0]-1
    a = ax[irow][icol]
    fname=df.iloc[i][analyze]
    poDF=pd.read_csv(fname,sep=" ",header=None)
    poDF=poDF.dropna(axis=1)
    data = poDF.replace(0,1e-10)
    data = np.abs(np.fft.fftshift(np.fft.fft2(data)))
    #data=data/data.sum().sum()
    min_data = data.min().min()
    if (min_data == 0):
        min_data = 1
    max_data = data.max().max()
    log_norm = LogNorm(vmin=min_data, vmax=max_data)
    cbar_ticks = [10**i for i in 
                  range(int(np.floor(np.log10(min_data))),
                        1 + int(np.ceil(np.log10(max_data))))]
    sns.heatmap(poDF,#norm=log_norm,
                cmap=cm.viridis, ax=a)#,
                #cbar_kws={"ticks": cbar_ticks})

In [ ]:
fig,(a1,a2,a3) = plt.subplots(1,3,figsize=(12,3))
sns.heatmap(poDF,##norm=log_norm,
            cmap=cm.viridis,ax=a1)#,
fft_data = np.fft.fftshift(np.fft.fft2(poDF))
data = np.abs(fft_data)
sns.heatmap(data,#norm=log_norm,
            cmap=cm.viridis,ax=a3)# cbar_kws={"ticks": cbar_ticks})
data = np.abs(np.fft.ifft2(np.fft.ifftshift(fft_data)))
sns.heatmap(data,##norm=log_norm,
            cmap=cm.viridis,ax=a2)#,
            #cbar_kws={"ticks": cbar_ticks})

In [ ]:
data